# Convolutional Neural Network

### Importing the libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
tf.__version__

'2.12.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
%cd gdrive/MyDrive

/content/gdrive/MyDrive


In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set =  train_datagen.flow_from_directory('dataset',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 274 images belonging to 2 classes.


### Preprocessing the Test set

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset',
                                            target_size = (64, 64),
                                            batch_size = 16,
                                            class_mode = 'binary')

Found 274 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [ ]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [ ]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [ ]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [ ]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
# cnn.fit(x = training_set, validation_data = test_set, epochs = 200)
cnn.fit(x = training_set, validation_data = test_set, epochs = 20)

## Part 4 - Making a single prediction

In [ ]:
!pip install flask-ngrok

In [ ]:
!pip install pyngrok==4.1.1
!ngrok authtoken 1hmRRl6ZYjxLqCode30MaVQk1kx_6DjEkRGLotgKYSGJ1zCGu

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

def predict(imagefile):
    print(type(imagefile))
    
    imagefile.save('image.jpg')
    test_image = image.load_img('image.jpg', target_size = (64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = cnn.predict(test_image)
    training_set.class_indices
    if result[0][0] == 1:
      prediction = 'Normal'
    else:
      prediction = 'Dysgraphia'
    return prediction

In [ ]:
from flask import Flask, request, render_template
from flask import jsonify
from flask_ngrok import run_with_ngrok

In [ ]:
from flask import Flask
app = Flask(__name__)
run_with_ngrok(app)


@app.route('/predict', methods = ['POST'])
def getResult():
    if(request.method == 'POST'):
        imagefile = request.files['image']
        res = predict(imagefile)
        return jsonify({'result': res})


@app.route("/")
def hello_world():
    return "<p>hello world this is Dysgraphia Detector </p>"

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://b82c-35-243-174-111.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [09/May/2023 18:30:55] "POST / HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [09/May/2023 18:31:25] "POST / HTTP/1.1" 405 -


<class 'werkzeug.datastructures.file_storage.FileStorage'>
1/1 [==============================] - 0s 33ms/step


INFO:werkzeug:127.0.0.1 - - [09/May/2023 18:31:37] "POST /predict HTTP/1.1" 200 -
